In [1]:
import numpy as np

from qibo import Circuit, gates, hamiltonians
from qibo.hamiltonians import XXZ
from qibo.models import VQE

# 定义量子比特数和层数
nqubits = 6
nlayers  = 4

# 创建变分量子电路
circuit = Circuit(nqubits)
for l in range(nlayers):
    # 添加旋转门 RY
    circuit.add(gates.RY(qubit, theta=0.0) for qubit in range(nqubits))
    # 添加控制相位门 CZ，作用于偶数索引的比特对
    circuit.add(gates.CZ(qubit, qubit + 1) for qubit in range(0, nqubits - 1, 2))
    # 再次添加旋转门 RY
    circuit.add(gates.RY(qubit, theta=0.0) for qubit in range(nqubits))
    # 添加控制相位门 CZ，作用于奇数索引的比特对
    circuit.add(gates.CZ(qubit, qubit + 1) for qubit in range(1, nqubits - 2, 2))
    # 添加控制相位门 CZ，作用于首尾比特对
    circuit.add(gates.CZ(0, nqubits - 1))
# 最后一层的旋转门 RY
circuit.add(gates.RY(qubit, theta=0.0) for qubit in range(nqubits))

# 创建 XXZ 哈密顿量
hamiltonian = XXZ(nqubits=nqubits)

# 创建 VQE 模型
vqe = VQE(circuit, hamiltonian)

# 从随机初始参数开始优化变分参数
initial_parameters = np.random.uniform(0, 2 * np.pi, nqubits * (2 * nlayers + 1))

# 使用 BFGS 方法进行参数优化
best, params, extra = vqe.minimize(initial_parameters, method='BFGS', compile=False)

[Qibo 0.2.21|INFO|2025-10-11 17:21:15]: Using qibojit (numba) backend on /CPU:0


In [5]:
circuit.draw()

0:     ─RY─o─────RY─────o─RY─o─────RY─────o─RY─o─────RY─────o─RY─o─────RY──── ...
1:     ─RY─Z─────RY─o───|─RY─Z─────RY─o───|─RY─Z─────RY─o───|─RY─Z─────RY─o── ...
2:     ─RY───o───RY─Z───|─RY───o───RY─Z───|─RY───o───RY─Z───|─RY───o───RY─Z── ...
3:     ─RY───Z───RY───o─|─RY───Z───RY───o─|─RY───Z───RY───o─|─RY───Z───RY───o ...
4:     ─RY─────o─RY───Z─|─RY─────o─RY───Z─|─RY─────o─RY───Z─|─RY─────o─RY───Z ...
5:     ─RY─────Z─RY─────Z─RY─────Z─RY─────Z─RY─────Z─RY─────Z─RY─────Z─RY──── ...

0: ... ─o─RY─
1: ... ─|─RY─
2: ... ─|─RY─
3: ... ─|─RY─
4: ... ─|─RY─
5: ... ─Z─RY─


In [2]:
best

np.float64(-9.472135954396322)

In [3]:
params

array([ 5.11582895e+00,  4.29607105e+00,  4.35510368e+00,  4.90610531e+00,
        6.11457063e-01, -3.49007760e-02,  5.52779268e+00,  2.41336321e+00,
        6.77641873e-01,  5.40952864e+00,  2.56571843e+00,  1.27327569e+00,
        4.71460181e+00,  3.14159234e+00,  1.32331413e+00,  6.70101824e+00,
        3.16717945e+00,  2.20525911e+00,  5.79944611e-06,  3.07000755e+00,
        7.27130360e+00,  4.64988958e+00,  2.00345345e+00,  3.01553589e+00,
        4.71523402e+00,  1.71579968e+00,  2.72441774e+00,  2.65674409e+00,
        3.77683764e+00,  3.14279166e+00,  4.71239652e+00, -9.36672696e-01,
        5.41013641e+00,  1.26460849e+00,  2.21014911e+00,  1.57081356e+00,
        3.54080046e+00,  4.67129062e+00, -5.45631449e-01,  3.61543428e+00,
        4.76930314e+00,  4.47909953e+00,  3.11111035e+00,  1.57080297e+00,
        4.80067803e+00,  3.96795691e+00,  4.71237767e+00,  3.68730172e+00,
        1.57078357e+00,  1.57078145e+00,  3.14156014e+00,  3.14154005e+00,
       -1.57082231e+00,  

In [4]:
extra

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -9.472135954396322
        x: [ 5.116e+00  4.296e+00 ... -1.571e+00  4.712e+00]
      nit: 319
      jac: [ 1.431e-06  1.907e-06 ...  2.384e-07  1.669e-06]
 hess_inv: [[ 3.056e+02  2.949e+02 ...  3.587e-01 -2.437e+00]
            [ 2.949e+02  2.858e+02 ...  3.320e-01 -2.371e+00]
            ...
            [ 3.587e-01  3.320e-01 ...  2.530e+00  2.288e+00]
            [-2.437e+00 -2.371e+00 ...  2.288e+00  3.152e+00]]
     nfev: 18810
     njev: 342

### 变分量子本征求解器（VQE）在Qibo中的实现详解

本指南将详细介绍如何使用Qibo框架实现变分量子本征求解器（VQE），并解释每个关键函数的输入与输出。

#### 1. 导入必要的库
```python
import numpy as np
from qibo import Circuit, gates, hamiltonians
from qibo.models import VQE
```
- **numpy**: 用于数值计算，特别是数组操作。
- **qibo**: 量子计算框架，提供构建量子电路、定义量子门和哈密顿量的工具。

#### 2. 定义量子比特数和层数
```python
nqubits = 6
nlayers = 4
```
- **nqubits**: 量子比特的数量，决定了系统的大小。
- **nlayers**: 变分电路的层数，影响优化参数的数量和电路的表达能力。

#### 3. 创建变分量子电路
```python
circuit = Circuit(nqubits)
for l in range(nlayers):
    circuit.add(gates.RY(qubit, theta=0.0) for qubit in range(nqubits))
    circuit.add(gates.CZ(qubit, qubit + 1) for qubit in range(0, nqubits - 1, 2))
    circuit.add(gates.RY(qubit, theta=0.0) for qubit in range(nqubits))
    circuit.add(gates.CZ(qubit, qubit + 1) for qubit in range(1, nqubits - 2, 2))
    circuit.add(gates.CZ(0, nqubits - 1))
circuit.add(gates.RY(qubit, theta=0.0) for qubit in range(nqubits))
```
- **Circuit(nqubits)**: 创建一个包含指定数量量子比特的量子电路。
- **gates.RY(qubit, theta=0.0)**: 添加旋转门RY到电路，`theta`为旋转角度，初始设为0.0。
- **gates.CZ(qubit1, qubit2)**: 添加控制相位门CZ到电路，作用于指定的量子比特对。

#### 4. 创建XXZ哈密顿量
```python
hamiltonian = XXZ(nqubits=nqubits)
```
- **XXZ(nqubits=nqubits)**: 定义一个XXZ哈密顿量，用于描述系统的能量。

#### 5. 创建VQE模型
```python
vqe = VQE(circuit, hamiltonian)
```
- **VQE(circuit, hamiltonian)**: 创建一个VQE实例，结合定义的量子电路和哈密顿量。

#### 6. 初始化参数并优化
```python
initial_parameters = np.random.uniform(0, 2 * np.pi, nqubits * (2 * nlayers + 1))
best, params, extra = vqe.minimize(initial_parameters, method='BFGS', compile=False)
```
- **initial_parameters**: 随机生成初始参数，用于电路中旋转门的角度。
- **vqe.minimize()**: 使用指定的优化方法（如BFGS）最小化能量，返回最优能量值、对应的参数和其他额外信息。

#### 7. 输出结果
```python
circuit.draw()
print(best)
print(params)
print(extra)
```
- **circuit.draw()**: 绘制量子电路图。
- **best**: 最优能量值。
- **params**: 对应最优能量的参数集。
- **extra**: 优化过程中产生的额外信息，如迭代次数等。

通过上述步骤，您可以在Qibo中实现一个基本的VQE模型，用于求解指定哈密顿量的基态能量。每个函数和步骤都精心设计，以确保模型的有效性和效率。